In [1]:
!pip install -U "huggingface_hub[cli]" -qqq
!pip install hf_transfer -qqq

^C


In [2]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [3]:
!pip install transformers accelerate bitsandbytes  -qqq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.8 MB/s eta 0:00:00


In [4]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)


In [5]:
#from google.colab import drive
#drive.mount("/content/drive")

In [6]:
"""cache_dir = "drive/MyDrive/hf_cache"
os.environ["HF_HOME"]        = cache_dir
os.environ["TRANSFORMERS_CACHE"] = cache_dir
os.environ["HF_DATASETS_CACHE"]  = cache_dir"""


"""del os.environ["HF_HOME"]
del os.environ['TRANSFORMERS_CACHE']
del os.environ['HF_DATASETS_CACHE']"""

'del os.environ["HF_HOME"]\ndel os.environ[\'TRANSFORMERS_CACHE\']\ndel os.environ[\'HF_DATASETS_CACHE\']'

In [7]:
model_id = "google/gemma-3-12b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"
)
# Write Your Code Here
tokenizer = AutoTokenizer.from_pretrained(
    model_id, use_fast=True
)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #cache_dir=cache_dir,
    device_map="auto",
    quantization_config=bnb_config,
    attn_implementation="eager",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [8]:
from peft import PeftModel

lora_model = PeftModel.from_pretrained(base_model, "drive/MyDrive/Regna_scenarios/gemma-3-12b-it-finetuned")
lora_model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): Gemma3ForConditionalGeneration(
      (model): Gemma3Model(
        (vision_tower): SiglipVisionModel(
          (vision_model): SiglipVisionTransformer(
            (embeddings): SiglipVisionEmbeddings(
              (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
              (position_embedding): Embedding(4096, 1152)
            )
            (encoder): SiglipEncoder(
              (layers): ModuleList(
                (0-26): 27 x SiglipEncoderLayer(
                  (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                  (self_attn): SiglipAttention(
                    (k_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
                    (v_proj): lora.Linear4bit(
                      (base_layer): Linear4bit(in_features=1152, out_features=1152, bias=True)
                      (lora_dropout): ModuleDict(
                        (defau

In [9]:
lora_pipe= pipeline(
    "text-generation",
    model=lora_model, #.merge_and_unload(),
    tokenizer=tokenizer,
    return_full_text=False,
)
"""messages = [
  {"role": "user", "content": "hello there!"},
]
output = lora_pipe(messages, max_new_tokens=512, temperature=0.1)[0]['generated_text']
print("Fine-tuned model:\n")
print(output)"""

Device set to use cuda:0
The model 'PeftModel' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gem

'messages = [\n  {"role": "user", "content": "hello there!"},\n]\noutput = lora_pipe(messages, max_new_tokens=512, temperature=0.1)[0][\'generated_text\']\nprint("Fine-tuned model:\n")\nprint(output)'

In [10]:
!pip install fastapi uvicorn pyngrok -qqq
!pip install nest_asyncio -qqq

In [11]:
# def make_prompt(text):


In [15]:
from fastapi import FastAPI
#from pydantic import BaseModel
from pyngrok import ngrok
import nest_asyncio
nest_asyncio.apply()

public_url = ngrok.connect(8000).public_url
print("ngrok URL:", public_url)

# 2. FastAPI app definition
app = FastAPI()

#class Payload(BaseModel):
#    prompt: str

@app.get("/generate")
async def generate(prompt: str):
    print("\nstart\n")
    messages = [{"role": "user", "content": prompt}]
    output = lora_pipe(messages, max_new_tokens=512, temperature=0.1)[0]["generated_text"]
    print(output)
    return {"response": output}

# 3. Run the server
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
  CORSMiddleware,
  allow_origins=["*"],
  allow_methods=["*"],
  allow_headers=["*"],
)

import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)

ngrok URL: https://afdf025a9e01.ngrok-free.app


INFO:     Started server process [638]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



start

```json
{
  "utterance": "A dragon, you say? A brave undertaking indeed! For such a foe, I would suggest a Red Potion to increase your damage, and perhaps a few Healing Potions should you take a blow. Blue Potions might be useful if you rely on spells, and an Orange Potion could offer some protection against fire, though I wouldn't want to overcharge you.",
  "action": "continue_conversation"
}
```<END_OF_RESPONSE>
INFO:     5.211.169.166:0 - "GET /generate?prompt=SYSTEM:You%20are%20Apothecary%20Helene,%20an%20NPC%20in%20a%20medieval%20roleplaying%20game.%20You%20must%20stay%20strictly%20in%20character%20and%20NEVER%20mention%20you%E2%80%99re%20an%20AI.===%20CHARACTER%20PROFILE%20===Name:%20Apothecary%20HeleneDescription:%20A%20skilled%20herbalist%20who%20runs%20the%20Greenleaf%20Apothecary%20in%20Ravenford.Personality:%20feminine%20and%20Warm;%20she%20admires%20bravery%20but%20dislikes%20haggling.Relationships:%20Respected%20by%20local%20adventurers%20for%20her%20reliable%20re

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [638]
